In [1]:
import pandas as pd
from nltk.tokenize import MWETokenizer
from tqdm.notebook import tqdm
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bhanuprakash.d\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [2]:
metadata=pd.read_pickle("umls_refined_metadata.pkl")

In [6]:
mw_list = []
for k,v in metadata.items():
    for x in v:
        ws = x.split() 
        if len(ws) > 1:
            tws = tuple(ws)
            mw_list.append(tws)
        
mw_list = ordered_set(mw_list)
###################################
my_idx = {}
for i,w in enumerate(mw_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

7880881


In [7]:
import pickle
with open('mw_list.pkl','wb') as f:
    pickle.dump(mw_list,f)

In [8]:
with open('my_idx.pkl','wb') as f:
    pickle.dump(my_idx,f)

In [3]:
mw_list=pd.read_pickle('mw_list.pkl')
my_idx=pd.read_pickle('my_idx.pkl')

In [4]:
mwe = MWETokenizer(mw_list,separator=' ')

In [5]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [6]:
def NER_Data_list(sentences):
    
    
    all_item = []

    for i in tqdm(range(len(sentences))):
        print("sentence",i+1)
        word_ls = []
        for token in spans(sentences[i]):
            my_tuple = token[0]
            flag=False
            for k,v in metadata.items():
                if token[0] in v:
                    #word_ls.append(my_tuple)
                    flag=True
                    subwords = token[0].split()
                    pos_list = [nltk.pos_tag([w]) for w in subwords]
                    tag_list = ['I-'+k]*len(pos_list)
                    tag_list[0] = 'B-'+k 


                    for s,p,t in zip(subwords,pos_list,tag_list):           
                        if type(p) == list:
                            p = p[0][1]
                            #print('list')
                        new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                        all_item.append(new_item)
                    break
            if flag==False:  
                my_pos = nltk.pos_tag([my_tuple])[0][1]
                new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
                all_item.append(new_item)
    return all_item

In [5]:
import pandas as pd
data=pd.read_csv('English_Sentences.csv')

c:\users\bhanuprakash.d\.conda\envs\test\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
sentences=list(data.sentences.values)

In [14]:
listy=sentences[:100000]
#NER_Data_list(listy)

In [1]:
from NER_Data_Preparator import NER_Data_list as ner

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\bhanuprakash.d\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Pickles Loaded
mwe tokenizer loaded


In [ ]:
result=ner(listy)

  0%|          | 0/100000 [00:00<?, ?it/s]

In [12]:
df_result=pd.DataFrame(result).fillna(method="ffill")
df_result[:10]

,Sentence #,Tag,Word,POS
0,1,B-Population Group,person,NN
1,1,O,to,TO
2,1,O,another,DT
3,1,O,and,CC
4,1,B-Functional Concept,causes,NNS
5,1,B-Sign or Symptom,fever,NN
6,1,O,",",","
7,1,B-Sign or Symptom,headache,NN
8,1,O,",",","
9,1,O,and,CC


In [13]:
len(data)

428441